<a href="https://colab.research.google.com/github/Robinwen9909/QUANTAXIS/blob/master/%E2%80%9Ccamera_basics_ipynb%E2%80%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NOTEBOOK_HEADER-->
This notebook contains material from the textbook [Multiple View Geometry in computer Vision](https://books.google.com.sg/books/about/Multiple_View_Geometry_in_Computer_Visio.html?id=si3R3Pfa98QC&source=kp_book_description&redir_esc=y) and lecture notes [Standford CS231A ](http://web.stanford.edu/class/cs231a/);

Written by Ding Man.

**Please install the necessary packages to run the code**

In [ ]:
%pip install scipy numpy opencv-python tqdm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Camera basics code demo

##Table of contents:
---
**1.   Camera basics**




## 1 Camera basics


---


**1.1 Camera Matrix Model**

**1.2 Camera calibration**

**1.3 Camera Distortion**

**1.4 Code demo 1: Camera calibration for distortion correction**



### ***1.1 Camera Matrix Model***

Camera Matrix Model describes a set of important parameters that affect how a **world point $P$ is mapped to image coordinates $P'$**.










#### ***1.1.1 Intrinsic parameters***

*   Translation vector $[c_x\ c_y]^T$: describe how 
image plane and digital image coordinates can differ by a translation
*   Change of units $k$ and $l$: digital images and image plane are represented in different units, one in pixels and one in physical measurements

*   Skewness $\theta$: caused by sensor manufacturing errors
*   [Distortion](https://en.wikipedia.org/wiki/Distortion_%28optics%29) (ignore for now, cover in 1.3)

Make use of [Homogeneous coordinates](https://en.wikipedia.org/wiki/Homogeneous_coordinates#:~:text=Any%20point%20in%20the%20projective,multiplied%20by%20a%20common%20factor), a point in 3D space and its image coordinates by a matrix vector relationship can be expressed as:
$$P' =
\begin{bmatrix}
\alpha & -\alpha\cot\theta & c_x\\
0 & \frac{\beta}{\sin\theta} & c_y\\
0 & 0 & 1
\end{bmatrix}\begin{bmatrix}
I & 0\\
\end{bmatrix}P = K\begin{bmatrix}
I & 0\\
\end{bmatrix}P$$

The matrix $K$ is often referred to as the **camera matrix**
$$ K = \begin{bmatrix}
x'\\y'\\z\\
\end{bmatrix}=\begin{bmatrix}
\alpha & -\alpha\cot\theta & c_x\\
0 & \frac{\beta}{\sin\theta} & c_y\\
0 & 0 & 1
\end{bmatrix}$$

Camera matrix $K$ has **5 degrees of freedom**: 2 for focal length, 2 for offset, and 1 for skewness. (assume no distortion). $K$ is unique and inherent to a given camera.

#### ***1.1.2 Extrinsic Parameters***
The above mapping is between a point $P'$ in the **3D camera reference system** to a point $P$ in the **2D image plane**. However, the information about the 3D world may be available in a different coordinate system. Hence, additional transformation captured by **rotation matrix $R$** and **translation vector $T$** is introduced to relate points from the **world reference system** to the **3D camera reference system**

Given a point in a world reference system $P_w$, we can compute its camera coordinates:
$$P'=K\begin{bmatrix}
R & T\\
\end{bmatrix}P_w=MP_w$$

Matrices $R$ and $T$ are known as the **extrinsic parameters** as they do not depend on the camera.

With extrinsic and intrinsic parameters $M$, mapping from a 3D point P in an arbitrary world
reference system to the image plane can hence be achieved.

In total, the projection matrix $M$ has **$11$ degrees of freedom**: 5 from the intrinsic camera matrix,
3 from extrinsic rotation, and 3 from extrinsic translation.


### ***1.2 Camera calibration***
**Purpose:** Estimate the extrinsic and intrinsic camera parameters.

**How:** Solve for the intrinsic camera matrix $K$ and
the extrinsic parameters $R, T$

**Setup:** Provide calibration rig which consists of a simple pattern, $i.e.$ checkerboard) with known dimensions. the rig defines our world reference frame with origin $O_w$ and axes $i_w,\ j_w,\ k_w$. From the rig's known pattern, we have known points
in the world reference frame $P_1,..., P_n$. Finding these points in the image we
take from the camera gives corresponding points in the image $p_1,..., p_n$
<figure>
<img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon/calib_setup.PNG' alt="Setup">
<figcaption>Figure1: The setup of an example calibration rig</figcaption></center>
</figure>

Hence, a linear system of equations from $n$ correspondences can be set up such that for each correspondence $P_i$, $p_i$ and camera matrix $M$ whose rows are $m_1,m_2,m_3$:

$$p_i=\begin{bmatrix}
u_i \\ v_i\\
\end{bmatrix}=MP_i=\begin{bmatrix}
\frac{m_1p_i}{m_3p_i} \\ \frac{m_2p_i}{m_3p_i}\\
\end{bmatrix}$$

which is equivalent of solving a pair of equations:
$$u_i(m_3P_i)-m_1P_i=0$$
$$v_i(m_3P_i)-m_2P_i=0$$

Here, there are $2$ constraints for solving the unknown parameters contained in $m$. From 1.1.2, we know that the **camera matrix $M$ has 11 unknown parameters**. This suggests that we need **at least $6$ correspondences** to solve this.

**Note:** Not all sets of $n$ correspondences will work. For example, if the points $Pi$ lie on the same plane, then the system will not be able to be solved. These unsolvable configurations of points are known as **degenerate configurations**.

In the real world applications (such as the demo in 1.4), we often use more than 6 points as measurements are often noisy. When $2n > 11$, the above homogeneous linear system is overdetermined. Therefore, it can be treated as a minimization problem and can be solved using [singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition).

### **1.3 Camera distortion**

So far, we have been working with ideal lenses which are free from any distortion. However, as seen before, real lenses can deviate from rectilinear projection, which require more advanced methods to un-distort the images. 

Two major kinds of distortion are **radial distortion** and **tangential distortion**.

Radial distortion causes straight lines to appear curved. Radial distortion becomes larger the farther points are from the center of the image, shown on the left. While tangential distortion occurs because the image-taking lense is not aligned perfectly parallel to the imaging plane. So, some areas in the image may look nearer than expected.

<figure>
<img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon/distortion.jpg' alt="Distortion illustration">
<figcaption>Figure2: radial distortion and tangential distortion</figcaption></center>
</figure>

- Radial distortion can be represented as follows:
$$x_{distorted}=x(1+k_1r^2+k_2r^4+k_3r^6)$$
$$y_{distorted}=y(1+k_1r^2+k_2r^4+k_3r^6)$$

- Tangential distortion can be represented as follows:
$$x_{distorted}=x+[2p_1xy+p_2(r^2+2x^2)]$$
$$y_{distorted}=y+[2p_2xy+p_1(r^2+2y^2)]$$

In total, distortion coefficients consist of five unknown parameters:
$$Distortion\ coefficients=(k_1\ k_2\ k_3\ p_1\ p_2)$$

In the next section, we will see how to find the distortion coefficients and correct the distortion using some sample images of a well defined pattern (e.g. a chess board).


### **1.4 Code demo: Camera calibration for distortion correction**

This code demo is written in [Python](https://www.python.org/) which uses pre-defined functions from [OpenCV](https://opencv.org/) library. The reference can be found [here](https://docs.opencv.org/master/dc/dbb/tutorial_py_calibration.html). 

The distorted images used in this demo were taken by a thermal-RGB camera for drone operations, the detail specs can be found [here](https://www.dji.com/sg/zenmuse-xt2). We will only focus on RGB images in this task. 

<figure>
<img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon//sample_rgb_distort.jpg' alt="sample distorted rgb image">
<figcaption>Figure2: Sample distorted rgb image</figcaption></center>
</figure>





Import packages

In [ ]:
import os
import glob
import tqdm
import cv2 as cv
import numpy as np

A function `get_camera_params` has been written for you to execute the above steps altogether. Instead of inputing just one image, we will use 3 RGB raw images inside folder `rgb_raw` to get a better estimation of camera parameters. 

Run the two code cells below. Note that it takes roughly 5 mins to complete the process. 

This function does the following:

*   Prepare 3D real world points (object points) and the corresponding 2D coordinates (image points) of these points in each image by finding pattern in chess board using [`cv.findChessboardCorners()`](https://docs.opencv.org/master/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a)

*   Increase the accuracy of the points using [`cv.cornerSubPix()`](https://docs.opencv.org/master/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e) and draw the pattern using [`cv.drawChessboardCorners()`](https://docs.opencv.org/master/d9/d0c/group__calib3d.html#ga6a10b0bb120c4907e5eabbcd22319022).
*   Calibrate the camera using [`cv.calibrateCamera()`](https://docs.opencv.org/master/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d) which returns the camera matrix $K$, distortion coefficients, rotation and translation vectors $R, T$
*   Calculate re-projection error which gives a good estimation of how exact the found parameters are. This is done throught transforming the object point to image point using [`cv.projectPoints()`](https://docs.opencv.org/master/d9/d0c/group__calib3d.html#ga1019495a2c8d1743ed5cc23fa0daff8c) and calculate the absolute norm between the transformation and the corner finding algorithm.



In [ ]:
def get_camera_params(number_rows: int,number_cols: int,input_folder: str):
    '''
    get camera parameters based on a set of chessboard images (>10 images)

    Parameters
    ----------
    number_rows, number_cols : int, point number of rows/columns of chessboard
    input_folder : str
    '''
    # findChessBoard flags
    find_flags = cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_NORMALIZE_IMAGE + \
    			        cv.CALIB_CB_FILTER_QUADS + cv.CALIB_CB_FAST_CHECK
       
    # Define termination criteria for corner finding algorithm
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
       
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((number_cols*number_rows, 3), np.float32)
    objp[:,:2] = np.mgrid[0:number_rows, 0:number_cols].T.reshape(-1,2)
    
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    
    # read images
    images = glob.glob(os.path.join(input_folder, '*.jpg'))+ \
              glob.glob(os.path.join(input_folder, '*.JPG'))

    fail_all = True
    for fname in tqdm.tqdm(images):
  
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)    

        # Find the chess board corners
        ret,corners = cv.findChessboardCorners(gray,(number_rows,number_cols),
                                               find_flags)
    
        # If found, add object points, image points (after refining them)    
        if ret == True:
            fail_all = False
            objpoints.append(objp)
            # improve accuracy
            corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            imgpoints.append(corners2)
            
            # Draw the corners 
            cv.drawChessboardCorners(img,(number_rows,number_cols),corners2,ret)

            # save images with corner points
            raw_img = os.path.split(fname)     
            img_name = 'draw_'+raw_img[-1]   
            cv.imwrite(os.path.join(input_folder, img_name), img)
           
            # display succeed or fail to find corners
            print(fname, ': succeed to find corner')
        else:
            print(fname, ': fail')
        
    # Calibration camera to find intrinsic, extrinsic parameters
    if fail_all == False:
        imageSize = gray.shape[::-1]
        err,KK,distCoeffs,rvecs,tvecs = cv.calibrateCamera(objpoints,imgpoints,
                                                           imageSize,None,None)

        # Calculate re-projection error
        total_error = 0
        for i in range(len(objpoints)):
            imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i],
                                             tvecs[i], KK, distCoeffs)
            error = cv.norm(imgpoints[i],imgpoints2,cv.NORM_L2)/len(imgpoints2)
            total_error += error
        mean_err = total_error/len(objpoints)
        return mean_err, KK, distCoeffs, rvecs, tvecs
    else:
        return

**TO DO:** Upload the sample image folder *rgb_raw* to the Files session. Run the cell below. 

After execution, check the same folder. Open the images with name "draw_*", you can observe the detected chessboard corners. 

In [ ]:
image_folder = "/content/rgb_raw"
rows = 9
columns = 7
err,K,distCoeffs,rvecs,tvecs = get_camera_params(rows, columns, image_folder)

 33%|███▎      | 1/3 [00:01<00:02,  1.08s/it]

/content/rgb_raw/DJI_0036.jpg : succeed to find corner


 67%|██████▋   | 2/3 [00:01<00:00,  1.34it/s]

/content/rgb_raw/DJI_0042.jpg : succeed to find corner


100%|██████████| 3/3 [00:30<00:00, 10.20s/it]

/content/rgb_raw/DJI_0038.jpg : succeed to find corner


In [ ]:
print("Mean re-projection error:", err)
print("----------------------------------------------------------")
print("Camera matrix: ", K)
print("----------------------------------------------------------")
print("Distortion coefficients: ", distCoeffs)
print("----------------------------------------------------------")
print("Rotation matrix", rvecs)
print("----------------------------------------------------------")
print("Translation matrix", tvecs)

Mean re-projection error: 0.12223989083878928
----------------------------------------------------------
Camera matrix:  [[1.71329380e+03 0.00000000e+00 1.93968085e+03]
 [0.00000000e+00 1.70873472e+03 1.61324143e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
----------------------------------------------------------
Distortion coefficients:  [[-0.06274633  0.0047193  -0.00281447 -0.00221993 -0.00010191]]
----------------------------------------------------------
Rotation matrix (array([[0.0249691 ],
       [0.02844343],
       [1.59404469]]), array([[-0.0588217 ],
       [ 0.06098872],
       [ 2.87823615]]), array([[0.03187022],
       [0.03462531],
       [1.58949149]]))
----------------------------------------------------------
Translation matrix (array([[ 7.92215584],
       [-5.06001038],
       [ 7.8458748 ]]), array([[ 9.4203175 ],
       [-0.69900003],
       [ 8.1042969 ]]), array([[ 9.35499736],
       [-4.7816685 ],
       [ 7.78533678]]))


**Add-on challenge:** It is possible to calibrate thermal camera as well using a spcially fabricated chessboard made of aluminium foil. Unlike RGB images, image pre-processing is required to extract the temperature and to set thresholds for the thermal images, as shown below. I have provided a folder named `IR_raw` with 12 pre-processed thermal images. See if you can calibrate the thermal camera by running the function `get_camera_params`. *Hint*: change the size of the chessboard. 

<img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon/sample_thermal_distort.JPG' width="425"/> <img src='https://raw.githubusercontent.com/DINGMAN17/Learning_material/main/3D_recon/sample_thermal_theshold.JPG' width="425"/> 



Instruction: fill in the correct value for the two variables `number_rows` and `number_cols`. Uncomment and run the cell below. You will be able to get the parameters for the thermal camera. 

In [ ]:
number_rows = 5 
number_cols = 4
input_folder = '/content/ir_raw'
err,K,distCoeffs,rvecs,tvecs = get_camera_params(number_rows,number_cols,input_folder)

 12%|█▎        | 1/8 [00:00<00:00,  8.82it/s]

/content/ir_raw/DJI_0425_R_thermal_gray_r.JPG : succeed to find corner


 38%|███▊      | 3/8 [00:00<00:01,  4.56it/s]

/content/ir_raw/DJI_0427_R_thermal_gray_r.JPG : fail
/content/ir_raw/DJI_0367_R_thermal_gray_r.JPG : succeed to find corner


 50%|█████     | 4/8 [00:00<00:00,  5.30it/s]

/content/ir_raw/DJI_0375_R_thermal_gray_r.JPG : succeed to find corner
/content/ir_raw/DJI_0699_R_thermal_gray.JPG : succeed to find corner


 88%|████████▊ | 7/8 [00:01<00:00,  5.02it/s]

/content/ir_raw/DJI_0363_R_thermal_gray_r.JPG : fail
/content/ir_raw/DJI_0199_R_thermal_gray.JPG : fail


100%|██████████| 8/8 [00:01<00:00,  4.46it/s]

/content/ir_raw/DJI_0407_R_thermal_gray_r.JPG : fail


In [ ]:
#if you are interested to check out the camera parameters, uncomment the parameters to print them out.

print("Re-projection error:", err)
print("Camera matrix: ", K)
print("Distortion coefficients: ", distCoeffs)
print("Rotation matrix", rvecs)
print("Translation matrix", tvecs)

Re-projection error: 0.10474368517916596
Camera matrix:  [[1.16103201e+03 0.00000000e+00 3.51533918e+02]
 [0.00000000e+00 1.17431329e+03 2.44979718e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:  [[-5.61890291e-01  4.93011749e+01 -1.22900969e-02  2.16887960e-02
  -7.20001064e+02]]
Rotation matrix (array([[ 0.48554362],
       [-0.04220577],
       [-0.0604111 ]]), array([[ 0.65784413],
       [-0.03886907],
       [-0.08328889]]), array([[ 0.65850537],
       [-0.05847263],
       [-0.03006971]]), array([[-0.95920619],
       [-0.09938894],
       [-0.14894696]]))
Translation matrix (array([[-1.67091389],
       [ 1.71468565],
       [23.84734057]]), array([[-0.73132156],
       [-2.67497355],
       [22.33402575]]), array([[-3.21012088],
       [-2.75250986],
       [22.38738669]]), array([[-3.45976145],
       [ 0.35016234],
       [21.57875381]]))


**Your turn:** 

Previously I have provided the images for you to play around with the code. Now it's your turn to take the images of chessboard!

Before rushing to the chessboard, have you thought about the questions below?

1.   *How many images shall I take?*
2.   *How to take the images?*
3.   *What are the factors I need to consider when taking the images? (angle of the chessboard? contrast?...)*
4.   *Anything else?*








In [ ]:
image_folder = "/content/WeTeam" 
err, K, distCoeffs, rvecs, tvecs = get_camera_params(9,7, image_folder)
print("Re-projection error:", err)
print("Camera matrix: ", K)
print("Distortion coefficients: ", distCoeffs)
print("Rotation matrix", rvecs)
print("Translation matrix", tvecs)

 33%|███▎      | 1/3 [00:01<00:03,  1.77s/it]

/content/WeTeam/Picture B.jpg : succeed to find corner


 67%|██████▋   | 2/3 [00:02<00:01,  1.19s/it]

/content/WeTeam/Picture C.jpg : succeed to find corner


100%|██████████| 3/3 [00:03<00:00,  1.11s/it]

/content/WeTeam/Picture A.jpg : succeed to find corner
Re-projection error: 0.37853310989642175
Camera matrix:  [[7.30626967e+03 0.00000000e+00 1.06023671e+03]
 [0.00000000e+00 7.27854775e+03 1.88798946e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:  [[ 6.37184075e-01 -8.80035444e+00 -6.95405354e-03 -2.18551178e-02
   6.02373199e+01]]
Rotation matrix (array([[-0.41605973],
       [-0.42541613],
       [ 1.34408766]]), array([[ 0.02554528],
       [-0.01065901],
       [ 1.57465402]]), array([[0.60288993],
       [0.49264253],
       [1.65067194]]))
Translation matrix (array([[ 3.82549111],
       [-5.192425  ],
       [32.09220635]]), array([[ 4.90273735],
       [-3.75301603],
       [25.96229948]]), array([[ 3.46940411],
       [-2.94914161],
       [22.82151541]]))


Optional: Distortion correction for sample images

In [ ]:
# read images
images = glob.glob('/content/rgb_raw/DJI*.jpg')
for fname in images: 
    targetImage = cv.imread(fname)
    
    # camera array
    targetImageSize = targetImage.shape[:2][::-1]
    alpha = 1  # 1:black
    newKK, roiSize = cv.getOptimalNewCameraMatrix(K, distCoeffs, targetImageSize, alpha, targetImageSize)
    
    # distotion correct map
    mapX, mapY = cv.initUndistortRectifyMap(K, distCoeffs, None, newKK, targetImageSize, cv.CV_32FC1)
    
    # distortion
    undistortedImage1 = cv.remap(targetImage, mapX, mapY, cv.INTER_LINEAR)
    
    # remove black parts
    x, y, w, h = roiSize
    undistortedImage2 = undistortedImage1[y:y+h, x:x+w]
    
    # save images
    img_name = os.path.split(fname)
    os.makedirs('/content/undistort', exist_ok=True)
    os.makedirs('/content/undistort_crop', exist_ok=True)
    cv.imwrite('/content/undistort/'+ img_name[-1], undistortedImage1)
    cv.imwrite('/content/undistort_crop/'+ img_name[-1], undistortedImage2)
    
    # display the result
    print(fname + ': undistorted and cropped')

/content/rgb_raw/DJI_0036.jpg: undistorted and cropped
/content/rgb_raw/DJI_0042.jpg: undistorted and cropped
/content/rgb_raw/DJI_0038.jpg: undistorted and cropped
